# Text To Script Proof Of Concept

This notebook will walk through the implementation, of a creative AI tool which helps writers turn text into workable scripts instantly. 

This process can be split up into 3 parts:
1. Input sentence splitting: The input text will be split into sentences in order for a emotion classifier to be able to tag every sentence seperately.
2. Sentence emotion tagging: The split sentences will be tagged with an emotion and a confidence score.
3. Combining resulting sentences as an input for a LLM: The resulting tagged sentences will be combined together again in order to be used as an input to a LLM
4. Generating a script using a LLM: The LLM will be prompted to write a script based on the input text which now has emotion tags after every sentence in order to better capture the emotion in a line for the actors to use.



## Input sentence splitting


In [1]:
# This cell sets up the functions for sentence splitting
import nltk 
nltk.download("punkt_tab")

from nltk.tokenize import sent_tokenize

def split_sentences(text):
    return sent_tokenize(text)

# Text from Charles Dickens' American Notes
example_text = """
‘Dinner, if you please,’ said I to the waiter.

‘When?’ said the waiter.

‘As quick as possible,’ said I.

‘Right away?’ said the waiter.

After a moment’s hesitation, I answered ‘No,’ at hazard.

‘_Not_ right away?’ cried the waiter, with an amount of surprise that
made me start.

I looked at him doubtfully, and returned, ‘No; I would rather have it in
this private room.  I like it very much.’
"""

sentences = split_sentences(example_text)


for i, sentence in enumerate(sentences):
    print (f"Sentence {i+1}: {sentence}")

Sentence 1: 
‘Dinner, if you please,’ said I to the waiter.
Sentence 2: ‘When?’ said the waiter.
Sentence 3: ‘As quick as possible,’ said I.
Sentence 4: ‘Right away?’ said the waiter.
Sentence 5: After a moment’s hesitation, I answered ‘No,’ at hazard.
Sentence 6: ‘_Not_ right away?’ cried the waiter, with an amount of surprise that
made me start.
Sentence 7: I looked at him doubtfully, and returned, ‘No; I would rather have it in
this private room.
Sentence 8: I like it very much.’


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hazel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Emotion Tagging

To obtain the emotion of a given sentence, we used a model from hugging face. The model outputs a probability distribution across the following labels: sadness, anger, love, joy, fear, and surprise

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

Device set to use cpu
C:\Users\hazel\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'sadness', 'score': 0.0006792700733058155}, {'label': 'joy', 'score': 0.9959298968315125}, {'label': 'love', 'score': 0.0009452445083297789}, {'label': 'anger', 'score': 0.0018055211985483766}, {'label': 'fear', 'score': 0.0004111040325369686}, {'label': 'surprise', 'score': 0.00022885717044118792}]]


### Obtain single Emotion

For our purposes it is useful to use the label with the highest probability as the predicted emotion

In [3]:
def top_emotion(predicted):
    scores = predicted[0]  
    best_label = None
    best_score = -1
    
    for item in scores:
        label = item['label']
        score = item['score']
        if score > best_score:
            best_score = score
            best_label = label
            
    return best_label, best_score


label, score = top_emotion(prediction)
label, score

('joy', 0.9959298968315125)

### Labeling Sentences

In [4]:
def label_sentences(sentences, classifier):
    results = []
    for s in sentences:
        pred = classifier(s)
        label_pred, score_pred = top_emotion(pred)
        results.append((s, label_pred, score_pred))
    return [{"sentence": s, "emotion" : label, "probability" : score} for s, label, score in results]


### Example

In [5]:
labeled = label_sentences(sentences, classifier)
labeled

[{'sentence': '\n‘Dinner, if you please,’ said I to the waiter.',
  'emotion': 'anger',
  'probability': 0.9002645015716553},
 {'sentence': '‘When?’ said the waiter.',
  'emotion': 'anger',
  'probability': 0.5363867282867432},
 {'sentence': '‘As quick as possible,’ said I.',
  'emotion': 'joy',
  'probability': 0.9724896550178528},
 {'sentence': '‘Right away?’ said the waiter.',
  'emotion': 'anger',
  'probability': 0.5925739407539368},
 {'sentence': 'After a moment’s hesitation, I answered ‘No,’ at hazard.',
  'emotion': 'anger',
  'probability': 0.6762022972106934},
 {'sentence': '‘_Not_ right away?’ cried the waiter, with an amount of surprise that\nmade me start.',
  'emotion': 'joy',
  'probability': 0.34155404567718506},
 {'sentence': 'I looked at him doubtfully, and returned, ‘No; I would rather have it in\nthis private room.',
  'emotion': 'anger',
  'probability': 0.918006956577301},
 {'sentence': 'I like it very much.’',
  'emotion': 'joy',
  'probability': 0.90528434514999

## Generative Model

In [6]:
!pip install -q openai

### Without emotion prompting

In [12]:

from openai import OpenAI
import os
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def call_gpt(prompt, model="gpt-4o-mini", temperature=0.3, max_tokens=300, seed=42):
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional screenwriter"},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content


In [13]:
text = example_text

prompt = f"""
Convert the text into SCREENPLAY FORMAT. 
Text:
{text}
"""
print(call_gpt(prompt))


```plaintext
INT. RESTAURANT - NIGHT

A dimly lit, elegant restaurant. The sound of clinking cutlery and soft chatter fills the air. At a table, a PATRON, dressed smartly, gestures to a WAITER.

                        PATRON
          Dinner, if you please.

                        WAITER
          When?

                        PATRON
          As quick as possible.

                        WAITER
          Right away?

The PATRON hesitates, considering.

                        PATRON
          No.

The WAITER's eyes widen in surprise.

                        WAITER
          _Not_ right away?

The PATRON looks at him, unsure.

                        PATRON
          No; I would rather have it in this private room. 
          I like it very much.

The WAITER nods, still a bit taken aback, and heads off to fulfill the order.

FADE OUT.
```


### Using emotion tag information and more explicit instructions

In [14]:
def screenplay_prompt(labeled):
    sents_with_emotions = []
    
    for x in labeled:
        sentence = x["sentence"]
        emotion = x["emotion"]
        
        sents_with_emotions.append(
            f"{sentence} (Express the emotion '{emotion}' implicitly through subtext, tone, and action. Do NOT use emotion words.)"
        )
    
    joined = "\n".join(sents_with_emotions)

    return f"""
Rewrite the following passage into screenplay format.

Each line includes an emotion that must be conveyed IMPLICITLY.
Do NOT use explicit emotion words. Show emotion only via actions, pauses, tone, and subtext.

Guided passage:
{joined}

Output ONLY the screenplay.
"""


In [15]:
text = example_text
sentences = split_sentences(text)
labeled_sentences = label_sentences(sentences, classifier)
prompt = screenplay_prompt(labeled_sentences)
out = call_gpt(prompt, temperature=0.2, max_tokens=700)
print(out)


```screenplay
INT. RESTAURANT - EVENING

A dimly lit, upscale restaurant. The clinking of silverware and low murmurs fill the air. 

At a corner table, I sit, fingers tapping impatiently on the tablecloth. I catch the WAITERS' eye.

                         ME
               (voice steady, clipped)
          Dinner, if you please.

The WAITER, caught off guard, raises an eyebrow, his pen hovering above his notepad.

                         WAITER
          When?

I lean forward slightly, my jaw tightening as I maintain eye contact.

                         ME
               (tone sharp)
          As quick as possible.

The WAITER shifts uncomfortably, glancing at the bustling kitchen.

                         WAITER
          Right away?

I exhale slowly, a tight smile forming as I lean back, crossing my arms.

                         ME
               (after a beat)
          No.

The WAITER blinks, taken aback, his surprise evident as he leans in closer.

                        

### Labeled Emotions Used To Generate Response

In [16]:
labeled_sentences

[{'sentence': '\n‘Dinner, if you please,’ said I to the waiter.',
  'emotion': 'anger',
  'probability': 0.9002645015716553},
 {'sentence': '‘When?’ said the waiter.',
  'emotion': 'anger',
  'probability': 0.5363867282867432},
 {'sentence': '‘As quick as possible,’ said I.',
  'emotion': 'joy',
  'probability': 0.9724896550178528},
 {'sentence': '‘Right away?’ said the waiter.',
  'emotion': 'anger',
  'probability': 0.5925739407539368},
 {'sentence': 'After a moment’s hesitation, I answered ‘No,’ at hazard.',
  'emotion': 'anger',
  'probability': 0.6762022972106934},
 {'sentence': '‘_Not_ right away?’ cried the waiter, with an amount of surprise that\nmade me start.',
  'emotion': 'joy',
  'probability': 0.34155404567718506},
 {'sentence': 'I looked at him doubtfully, and returned, ‘No; I would rather have it in\nthis private room.',
  'emotion': 'anger',
  'probability': 0.918006956577301},
 {'sentence': 'I like it very much.’',
  'emotion': 'joy',
  'probability': 0.90528434514999